### **Multicopter design calculation**

In [1]:
import math
import numpy as np
import pandas as pd
import splinecloud_scipy as scsp
import matplotlib.pyplot as plt
from dataclasses import dataclass
%matplotlib inline 
from scipy.optimize import brentq
from scipy.optimize import fmin
from pprint import pprint
from copy import deepcopy
from scipy.optimize import differential_evolution, Bounds

### Load data from SplineCloud

In [2]:
from components_data import propellers, motors, battery_groups, quad_frame, Propeller, Motor, Battery

### Define Constants

Number of motors

In [3]:
n_motor = 4 ## quadcopter

Payload mass

In [4]:
m_payload = 1.5 ## [kg]

Hovering time

In [5]:
t_hover = 0.5 ## [hours]
t_hover_min = t_hover * 0.85

Maximum battery discharge ratio

In [6]:
k_discharge = 0.2

Hovering throttle range

In [7]:
# hov_throttle_range = np.linspace(0.4, 0.6, 11)
hov_throttle_range = np.linspace(0.4, 0.6, 5)
hov_throttle_range

array([0.4 , 0.45, 0.5 , 0.55, 0.6 ])

Gravity acceleration

In [8]:
g = 9.81 ## [kg * m/c^2]

Single cell voltage

In [9]:
cell_voltage = 3.7 ## [V]

Dynamic load factor

In [10]:
ks_dyn = 3

### Design calculation

Guess copter mass to the payload mass (initial value)

In [11]:
km_copter = 3.5

Total copter guess mass (with payload)

In [12]:
m_total = (km_copter + 1) * m_payload

Required hovering thrust per motor

In [13]:
F_hover = m_total * g / n_motor
print(F_hover, "[N]")

16.554375 [N]


#### **Motor selection**

In [14]:
def select_motor(motors, f_hover, throttle_range):
    selected_motor = None
    thrust_diff = math.inf
    for hov_throttle in throttle_range:
        for motor in motors:
            thrust = motor.thrust_vs_throttle.eval(hov_throttle)*0.001*g
            diff = abs(thrust - f_hover)
            if thrust_diff > diff:
                thrust_diff = diff
                selected_motor = deepcopy(motor)
                selected_motor.hov_throttle = hov_throttle

    return selected_motor

In [15]:
motor = select_motor(motors, F_hover, hov_throttle_range)
motor.name

'u7v2_kv420_22.2v_15x5cf'

#### **Battery selection**

In [16]:
def select_battery(voltage, current):
    n_cells = round(voltage/cell_voltage)
    capacity = (n_motor * current * t_hover) / (1-k_discharge)
    
    selected_battery = None
    total_weight = math.inf
    n_batteries = range(1, n_motor)
    for batgroup in battery_groups.keys():
        for cellgroup in battery_groups[batgroup].keys():
            snum = int(cellgroup[0])
            if snum == int(n_cells):
                candidates = battery_groups[batgroup][cellgroup]
                for nbat in n_batteries:
                    c = 1000*capacity/nbat  
                    bcaps = [i for i in range(len(candidates.data.index)) if candidates.data.iloc[i,3] >= c]
                    if not bcaps:
                        continue
                    closest_ind = bcaps[0]
                    battery_row = candidates.data.iloc[closest_ind,:]
                    w = battery_row.iloc[-1]
                    if w*nbat < total_weight:
                        total_weight = w*nbat
                        selected_battery = Battery(battery_row, nbat)
    
    return selected_battery

In [17]:
battery = select_battery(motor.voltage, motor.hover_current)
battery

Battery(name=HP-G830C10000S6, capacity=10000, s=6, number=3, total_weight=3699, total_capacity=30000)

#### **Frame selection**

In [18]:
def select_frame(propeller_diam):
    propeller_gap_ratio = 1.1
    frame = deepcopy(quad_frame)
    frame.select(propeller_diam * propeller_gap_ratio)
    
    return frame

In [19]:
frame = select_frame(motor.propeller.diameter)
frame.weight

670.116540933877

#### **Copter total mass recalculation**

In [20]:
m_copter = n_motor*(motor.weight + motor.propeller.weight) + battery.total_weight + frame.weight
m_copter

5733.116540933877

#### Refined copter reduced weight

In [21]:
km_copter = m_copter*1e-3/m_payload
km_copter

3.8220776939559182

#### **Design optimization**

In [22]:
def design_iteration(km_copter_in, iteration=False):
    m_total = (km_copter_in + 1) * m_payload
    f_hover = m_total*g / n_motor
    
    motor = select_motor(motors, f_hover, hov_throttle_range)
    battery = select_battery(motor.voltage, motor.hover_current)
    frame = select_frame(motor.propeller.diameter)
    
    if not all([motor, battery, frame]):
        if iteration:
            return km_copter_in*10
        else:
            return
    
    m_copter = n_motor*(motor.weight + motor.propeller.weight) + battery.total_weight + frame.weight
    
    km_copter_out = m_copter*1e-3/m_payload
    
    res = {'km_copter_in': km_copter_in, 
           'km_copter_out': km_copter_out,
           'km_copter_diff': km_copter_out - km_copter_in,
           'm_copter': m_copter,
           'motor': motor,
           'battery': battery,
           'frame': frame,
    }
    
    if iteration:
        return abs(km_copter_out - km_copter_in)
    else:
        return res

In [23]:
design_iteration(3.76)

#### **Find solutions in a range of initial guesses**

In [24]:
global_solutions = {}
for kmcop in np.linspace(1, 10, 19):
    converged = fmin(design_iteration, kmcop, args=(True,), full_output=True, disp=1)
    if converged[-1] == 0:
        km_sol = round(converged[0][0], 3)
        res = design_iteration(km_sol)
        print(res)
        if res and km_sol not in global_solutions.keys():
            global_solutions[km_sol] = res

Optimization terminated successfully.
         Current function value: 0.505496
         Iterations: 15
         Function evaluations: 30
{'km_copter_in': 1.09, 'km_copter_out': 1.5955349254829982, 'km_copter_diff': 0.5055349254829982, 'm_copter': 2393.302388224497, 'motor': Motor(name=u3_kv700_14.8v_12x4cf, voltage=14.8, kv=700, weight=128, propeller=Propeller(name=P12x4 Prop-2PCS/PAIR, diameter=12, width=4.0)), 'battery': Battery(name=HP-G830C5200S4, capacity=5200, s=4, number=3, total_weight=1284, total_capacity=15600), 'frame': QuadFrame(base=543.4954210408756, weight=481.3023882244969, arm_width=18.0 payload_height=153.0, bottom_compartment_height=90.0 top_compartment_height=35.5, plate_thickness=3.2550926960017876)}
Optimization terminated successfully.
         Current function value: 0.506942
         Iterations: 15
         Function evaluations: 30
{'km_copter_in': 1.513, 'km_copter_out': 2.2744997347042353, 'km_copter_diff': 0.7614997347042354, 'm_copter': 3411.749602056353, 

In [25]:
pprint(global_solutions)

{1.09: {'battery': Battery(name=HP-G830C5200S4, capacity=5200, s=4, number=3, total_weight=1284, total_capacity=15600),
        'frame': QuadFrame(base=543.4954210408756, weight=481.3023882244969, arm_width=18.0 payload_height=153.0, bottom_compartment_height=90.0 top_compartment_height=35.5, plate_thickness=3.2550926960017876),
        'km_copter_diff': 0.5055349254829982,
        'km_copter_in': 1.09,
        'km_copter_out': 1.5955349254829982,
        'm_copter': 2393.302388224497,
        'motor': Motor(name=u3_kv700_14.8v_12x4cf, voltage=14.8, kv=700, weight=128, propeller=Propeller(name=P12x4 Prop-2PCS/PAIR, diameter=12, width=4.0))},
 1.513: {'battery': Battery(name=HP-G830C4400S6, capacity=4400, s=6, number=2, total_weight=1070, total_capacity=8800),
         'frame': QuadFrame(base=753.5617601072148, weight=893.7496020563528, arm_width=21.200000000000003 payload_height=192.0, bottom_compartment_height=130.0 top_compartment_height=40.0, plate_thickness=4.121554612463704),
    

#### **Design verification**

In [26]:
def verify_solution(solution):
    ## verify thrust is enough to hover
    km, sol = solution
    
    motor, battery, frame = sol["motor"], sol["battery"], sol["frame"]
    
    thrust_verified = False
    copter_mass = 1e-3*(battery.total_weight + frame.weight +
        n_motor*(motor.weight + motor.propeller.weight)
    )
    
    total_copter_weight = (copter_mass + m_payload) * g
    thrust_vs_throttle = motor.thrust_vs_throttle

    max_throttle_data = max(thrust_vs_throttle.coeffs_x)
    min_throttle_data = min(thrust_vs_throttle.coeffs_x)
    
    max_hov_throttle = min(max(hov_throttle_range), max_throttle_data)
    min_hov_throttle = max(min(hov_throttle_range), min_throttle_data)
    
    max_copter_thrust = thrust_vs_throttle.eval(max_hov_throttle)*0.001*g*n_motor
    min_copter_thrust = thrust_vs_throttle.eval(min_hov_throttle)*0.001*g*n_motor
    
    if not (min_copter_thrust <= total_copter_weight <= max_copter_thrust):
        return
    
    ## verify hover time is sufficient
    
    ## find exact hover throttle
    throttle_root = lambda throt: thrust_vs_throttle.eval(throt)*0.001*g*n_motor - total_copter_weight
    hov_throttle = brentq(throttle_root, min_hov_throttle, max_hov_throttle)
    
    ## calculate hover time
    current = motor.current_vs_throttle.eval(hov_throttle)
    t_hov = battery.capacity*1e-3 * battery.number * (1-k_discharge) / (n_motor * current)

    if t_hov >= t_hover_min:
        solstat = {
            "solution": solution,
            "copter_mass": copter_mass,
            "total_copter_weight": total_copter_weight,
            "hov_throttle": hov_throttle,
            "t_hov": t_hov,
        }
    
        return solstat

**Verified global solutions**

In [27]:
verified_solutions = [s for s in map(verify_solution, global_solutions.items()) if s is not None]

#### **Optimal solution**

In [28]:
opt_sol = None
copter_mass = math.inf
for vs in verified_solutions:
    # pprint(s)
    if vs['copter_mass'] < copter_mass:
        copter_mass = vs['copter_mass']
        opt_sol = vs
opt_sol

{'solution': (2.055,
  {'km_copter_in': 2.055,
   'km_copter_out': 2.1487443606225844,
   'km_copter_diff': 0.09374436062258429,
   'm_copter': 3223.116540933877,
   'motor': Motor(name=u5_kv400_v22.2_15x5cf, voltage=22.2, kv=400, weight=195, propeller=Propeller(name=P15x5 Prop-2PCS/PAIR, diameter=15, width=5.0)),
   'battery': Battery(name=HP-G830C4400S6, capacity=4400, s=6, number=3, total_weight=1605, total_capacity=13200),
   'frame': QuadFrame(base=648.5285905740451, weight=670.116540933877, arm_width=19.0 payload_height=180.0, bottom_compartment_height=109.99999999999999 top_compartment_height=40.0, plate_thickness=3.6883236542327458)}),
 'copter_mass': 3.223116540933877,
 'total_copter_weight': 46.33377326656133,
 'hov_throttle': 0.5655484588298176,
 't_hov': 0.4713881630406551}

In [29]:
converged_de = differential_evolution(design_iteration, [[1.0, 5.0]], args=(True,))
converged_de

     fun: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 803
     nit: 50
 success: True
       x: array([2.8131664])

In [30]:
conv_sol = design_iteration(float(converged_de.x))
conv_sol

{'km_copter_in': 2.813166401370902,
 'km_copter_out': 2.813166401370902,
 'km_copter_diff': 0.0,
 'm_copter': 4219.749602056353,
 'motor': Motor(name=u7v2_kv280_24v_18x6.1cf, voltage=24, kv=280, weight=299, propeller=Propeller(name=P18x6.1 Prop-2PCS/PAIR, diameter=18, width=6.1)),
 'battery': Battery(name=HP-G830C5200S6, capacity=5200, s=6, number=3, total_weight=1878, total_capacity=15600),
 'frame': QuadFrame(base=753.5617601072148, weight=893.7496020563528, arm_width=21.200000000000003 payload_height=192.0, bottom_compartment_height=130.0 top_compartment_height=40.0, plate_thickness=4.121554612463704)}